# MODELO MACHINE LEARNING DE RECOMENDACION

#### Convoco librerias.

In [36]:
import pandas as pd
from sklearn.metrics.pairwise        import cosine_similarity
from sklearn.metrics.pairwise        import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

#### Cargo los dataframes que voy a usar.

In [37]:
path_games_csv = (r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\df_games.csv')
df_games = pd.read_csv(path_games_csv)
path_items_csv = (r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\df_items.csv')
df_items = pd.read_csv(path_items_csv)
path_reviews_csv = (r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\df_reviews.csv')
df_reviews = pd.read_csv(path_reviews_csv)
path_reviews_coments_csv = (r'D:\Users\Arnaldo\Desktop\SISTEMAS\SOYHENRY\CURSO\PROYECTOS\PROYECTO INDIVIDUAL I\RECURSADO\ARCHIVOS\DATAFRAMES\df_reviews_coments.csv')
df_reviews_coments = pd.read_csv(path_reviews_coments_csv)

In [38]:
# Se ccrea  un nuevo DataFrame llamado df_games_final que contiene solo las columnas relevantes
game_columns = ['item_id', 'app_name']
df_games_final = df_games[game_columns]

# Se hace un join entre df_games y df_UserItems utilizando 'item_id' como clave
df_joined = pd.merge(df_games_final, df_reviews_coments[['item_id', 'review']], on='item_id', how='inner')

In [39]:
#Se crea una muestra para el modelo
muestra = df_joined.head(20000)

In [40]:
# Se crea el modelo de machine learning con Scikit-Learn
tfidf = TfidfVectorizer(stop_words='english')
muestra=muestra.fillna("")

tdfid_matrix = tfidf.fit_transform(muestra['review'])
cosine_similarity = linear_kernel( tdfid_matrix, tdfid_matrix)

### Función recomendacion_juego()
Ingresando el id de producto (juego) recibiremos una lista con 5 juegos recomendados similares

In [41]:
# Se crea la funcion de recomendación de 5 juegos recomendados para el juego ingresado.
def recomendacion_juego(id: int):
    if id not in muestra['item_id'].values:
        return {'mensaje': 'No existe el id del juego'}
    
    titulo = muestra.loc[muestra['item_id'] == id, 'app_name'].iloc[0]
    idx = muestra[muestra['app_name'] == titulo].index[0]
    sim_cosine = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_cosine, key=lambda x: x[1], reverse=True)
    sim_ind = [i for i, _ in sim_scores[1:6]]
    sim_juegos = muestra['app_name'].iloc[sim_ind].values.tolist()
    return {f'juegos recomendados para {id}': list(sim_juegos)}

In [47]:
# Ejemplo de uso del modelo 1 (2300 - DOOM II)
print(recomendacion_juego(2300))

{'juegos recomendados para 2300': ["Garry's Mod", 'Undertale', "Garry's Mod", 'POSTAL 2', 'Ultimate Doom']}


In [48]:
# Ejemplo de uso del modelo 2 (21000 - LEGO Batman)
print(recomendacion_juego(21000))

{'juegos recomendados para 21000': ['Toribash', 'Doom 3: BFG Edition', 'Portal 2', 'AdVenture Capitalist', 'Portal 2']}


In [44]:
muestra

,item_id,app_name,review
0,282010.0,Carmageddon Max Pack,oh where do i start dos
1,70.0,Half-Life,advertenciaeste anãlisis contiene posibles spo...
2,70.0,Half-Life,if you own any half life but not this one that...
3,70.0,Half-Life,this game has had me hooked for over 4 years a...
4,70.0,Half-Life,this is by far my most favourite game i have e...
...,...,...,...
19995,365590.0,Tom Clancy’s The Division™,played 10 mins oobisoft ded gg no re
19996,365590.0,Tom Clancy’s The Division™,the division is like eating the same food ever...
19997,365590.0,Tom Clancy’s The Division™,ëëì íì¹íëëë ì êµ ë²í íë ìì ëííµìì¼ëëë íµì ì ì ...
19998,365590.0,Tom Clancy’s The Division™,totally awesome new experience for openworld r...
